In [3]:
from keras import Input, Model
from keras.constraints import max_norm
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, GlobalMaxPooling2D, Add, Activation, Lambda, \
    Concatenate, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, AveragePooling2D, DepthwiseConv2D, \
    SeparableConv2D
from tensorflow.keras import backend as K
import numpy as np
import scipy.io as sio
from numpy import swapaxes
from keras import utils as np_utils
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import scipy
import scipy.io as sio
import numpy as np 
#---------------------------------------------------------------------------
from DataAugment import SlideWindow
from Import import Import
from Shuffle import Shuffle
from FuseAttentionModel import *
from OthersModel import *

In [4]:
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))
def ShallowConvNet(nb_classes, Chans=3, Samples=4000, dropoutRate=0.5):
    input_shape = (Samples, Chans, 1)
    conv_filters = (25, 1)
    conv_filters2 = (1, Chans)
    pool_size = (45, 1)
    strides = (15, 1)
    axis = -1
    input_main = Input(input_shape)
    block1 = Conv2D(20, conv_filters,
                    input_shape=input_shape,
                    kernel_constraint=max_norm(2., axis=(0, 1, 2)))(input_main)
    block1 = Conv2D(20, conv_filters2, use_bias=False,
                    kernel_constraint=max_norm(2., axis=(0, 1, 2)))(block1)
    block1 = BatchNormalization(axis=axis, epsilon=1e-05, momentum=0.1)(block1)
    block1 = Activation(square)(block1)
    block1 = AveragePooling2D(pool_size=pool_size, strides=strides)(block1)
    block1 = Activation(log)(block1)
    block1 = Dropout(dropoutRate)(block1)
    flatten = Flatten()(block1)
    dense = Dense(nb_classes, kernel_constraint=max_norm(0.5))(flatten)
    softmax = Activation('softmax')(dense)
    model=Model(inputs=input_main, outputs=softmax)
    return model

In [ ]:
ans=[]
for q in range(1,5):
    data=[]
    mark=[]
    exptype='2'
    folder='C3C4CZ0-30Hz'
    subnum=8
    dataname=[]
    names1=locals()
    names2=locals()
    for i in range(1,9):
        datapath='E:\\BCI数据\\yhdata\\全数据处理\\'+folder+'\\Sub00'+str(i)+'\\'+exptype+'\\data.mat'
        markpath='E:\\BCI数据\\yhdata\\全数据处理\\'+folder+'\\Sub00'+str(i)+'\\'+exptype+'\\mark.mat'
        a=sio.loadmat(datapath)['data']
        b=sio.loadmat(markpath)['label']
        names1['data0%s' % (i)] = a
        names2['mark0%s' % (i)] = b
    data.append(data01)
    data.append(data02)

#     data.append(data04)
    data.append(data05)
    data.append(data06)
    data.append(data07)
    data.append(data08)
 
#     data.append(data011) 

#     data.append(data014)
#     data.append(data015)

#     data.append(data017)
#     data.append(data018)
#     data.append(data019)
#     data.append(data020)

#     data.append(data021)
#     data.append(data022)

#     data.append(data024)
#     data.append(data025)
#     data.append(data026)
#     data.append(data027)
#     data.append(data028)
#     data.append(data029)
    data=np.array(data)
    print(data.shape)


    mark.append(mark01)
    mark.append(mark02)

#     mark.append(mark04)
    mark.append(mark05)
    mark.append(mark06)
    mark.append(mark07)
    mark.append(mark08)

#     mark.append(mark011)

#     mark.append(mark014)
#     mark.append(mark015)

#     mark.append(mark017)
#     mark.append(mark018)
#     mark.append(mark019)
#     mark.append(mark020)

#     mark.append(mark021)
#     mark.append(mark022)

#     mark.append(mark024)
#     mark.append(mark025)
#     mark.append(mark026)
#     mark.append(mark027)
#     mark.append(mark028)
#     mark.append(mark029)
    mark=np.array(mark)
    print(mark.shape)

    from numpy import swapaxes
    data=swapaxes(data,1,3)

    from numpy import swapaxes
    from keras import utils as np_utils
    x_train=[]
    x_test=[]
    y_train=[]
    y_test=[]
    print('数据集形状：',data.shape)
    for i in range(data.shape[0]):
    ##数据打乱
        index=[i for i in range(data[i].shape[0])]
        np.random.shuffle(index)
        data_shu=data[i][index]
        mark_shu=mark[i][index]
    ##划分测试集训练集
        x_train.append(data_shu[:int(0.8*data_shu.shape[0])])
        x_test.append(data_shu[int(0.8*data_shu.shape[0]):])
        y_train.append(mark_shu[:int(0.8*mark_shu.shape[0])])
        y_test.append(mark_shu[int(0.8*mark_shu.shape[0]):]) 
    x_train=np.array(x_train)
    x_test=np.array(x_test)
    y_train=np.array(y_train)
    y_test=np.array(y_test)
    print('训练集测试集数据及标签形状：',x_train.shape,x_test.shape,y_train.shape,y_test.shape)
    ##创建总体训练测试集
    all_x_train=x_train[0]
    all_y_train=y_train[0]
    all_x_test=x_test[0]
    all_y_test=y_test[0]
    for i in range(1,x_train.shape[0]):
        all_x_train=np.concatenate([x_train[i],all_x_train])
        all_y_train=np.concatenate([y_train[i],all_y_train])
        all_x_test=np.concatenate([x_test[i],all_x_test])
        all_y_test=np.concatenate([y_test[i],all_y_test])
    print('所有被试训练集测试集形状：',all_x_train.shape,all_x_test.shape,all_y_train.shape,all_y_test.shape)
    ##总体训练集数据增强
    all_x_train,all_y_train=SlideWindow(all_x_train,all_y_train,4000,80)
    all_x_train=np.array(all_x_train)
    all_y_train=np.array(all_y_train)

    all_x_test,all_y_test=SlideWindow(all_x_test,all_y_test,4000,80)
    all_x_test=np.array(all_x_test)
    all_y_test=np.array(all_y_test)
    ##数据打乱
    index=[i for i in range(all_x_train.shape[0])]
    np.random.shuffle(index)
    all_x_train=all_x_train[index]
    all_y_train=all_y_train[index]
    index=[i for i in range(all_x_test.shape[0])]
    np.random.shuffle(index)
    all_x_test=all_x_test[index]
    all_y_test=all_y_test[index]
    print('所有被试训练集测试集增强后数据形状：',all_x_train.shape,all_y_train.shape,all_x_test.shape,all_y_test.shape)
    ##标准化
    # for i in range(all_x_train.shape[0]):
    #     mean=all_x_train[i,:,:].mean()
    #     std=all_x_train[i,:,:].std()
    #     all_x_train[i,:,:]=(all_x_train[i,:,:]-mean)/std
    # for i in range(all_x_test.shape[0]):
    #     mean=all_x_test[i,:,:].mean()
    #     std=all_x_test[i,:,:].std()
    #     all_x_test[i,:,:]=(all_x_test[i,:,:]-mean)/std

    ##个体训练集、测试集数据增强，打乱
    each_x_train=[]
    each_y_train=[]
    each_x_test=[]
    each_y_test=[]
    for i in range(x_train.shape[0]):
        x,y=SlideWindow(x_train[i],y_train[i],4000,10)
        x=np.array(x)
        y=np.array(y)
        index=[i for i in range(x.shape[0])]
        np.random.shuffle(index)
        x=x[index]
        y=y[index] 
        each_x_train.append(x)
        each_y_train.append(y)
    for i in range(x_test.shape[0]):
        x,y=SlideWindow(x_test[i],y_test[i],4000,10)
        x=np.array(x)
        y=np.array(y)
        index=[i for i in range(x.shape[0])]
        np.random.shuffle(index)
        x=x[index]
        y=y[index] 
        each_x_test.append(x)
        each_y_test.append(y)
    x_train=np.array(each_x_train)
    y_train=np.array(each_y_train)
    x_test=np.array(each_x_test)
    y_test=np.array(each_y_test)
    print('每个被试增强后数据数据形状:',x_train.shape,y_train.shape,x_test.shape,y_test.shape)
    ##标准化
    # for i in range(x_train.shape[0]):
    #     for j in range(x_train.shape[1]):
    #         mean=x_train[i,j,:,:].mean()
    #         std=x_train[i,j,:,:].std()
    #         x_train[i,j,:,:]=(x_train[i,j,:,:]-mean)/std
    # for i in range(x_test.shape[0]):
    #     for j in range(x_test.shape[1]):
    #         mean=x_test[i,j,:,:].mean()
    #         std=x_test[i,j,:,:].std()
    #         x_test[i,j,:,:]=(x_test[i,j,:,:]-mean)/std
    ##2DCNN数据格式更改
    all_x_train=all_x_train.reshape(all_x_train.shape[0],all_x_train.shape[1],all_x_train.shape[2],1)
    all_x_test=all_x_test.reshape(all_x_test.shape[0],all_x_test.shape[1],all_x_test.shape[2],1)
    x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],x_train.shape[3],1)
    x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],x_test.shape[3],1)
    all_y_train=np_utils.to_categorical(all_y_train,num_classes=2)
    all_y_test=np_utils.to_categorical(all_y_test,num_classes=2)
    y1=[]
    y2=[]
    for i in range(y_train.shape[0]):
        y=np_utils.to_categorical(y_train[i],num_classes=2)
        y1.append(y)
    for i in range(y_test.shape[0]):
        y=np_utils.to_categorical(y_test[i],num_classes=2)
        y2.append(y)
    y_train=np.array(y1)
    y_test=np.array(y2)
    print('全被试训练集,测试集，单个被试训练集，测试集形状：',all_x_train.shape,all_y_train.shape,all_x_test.shape,all_y_test.shape,'\n',
          x_train.shape,x_test.shape,y_train.shape,y_test.shape)

    SA=FAModel(nb_classes=2)
    all_x_train=np.swapaxes(all_x_train,2,3)
    all_x_test=np.swapaxes(all_x_test,2,3)
    x_train=np.swapaxes(x_train,3,4)
    x_test=np.swapaxes(x_test,3,4)
    SA.summary() 
    optim = Adam(lr = 0.001)
    SA.compile(loss = 'categorical_crossentropy', optimizer =optim, metrics = ['accuracy'])
    history = SA.fit(all_x_train, all_y_train,validation_data=(all_x_test,all_y_test), epochs=1, batch_size=16)  
#     SA.save("model.h5")
    fig=plt.figure(0)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    fig=plt.figure(1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    a=[]
    for i in range(x_test.shape[0]):
        each_person=SA.evaluate(x_test[i],y_test[i])
        a.append(each_person[1])
        print('第',str(i+1),'个被试的正确率为:',str(each_person[1]))
    ans.append(a)
for i in range(10):
    print(ans[i])

(6, 3, 5000, 80)
(6, 80, 1)
数据集形状： (6, 80, 5000, 3)
训练集测试集数据及标签形状： (6, 64, 5000, 3) (6, 16, 5000, 3) (6, 64, 1) (6, 16, 1)
所有被试训练集测试集形状： (384, 5000, 3) (96, 5000, 3) (384, 1) (96, 1)
所有被试训练集测试集增强后数据形状： (4992, 4000, 3) (4992, 1) (1248, 4000, 3) (1248, 1)
每个被试增强后数据数据形状: (6, 6464, 4000, 3) (6, 6464, 1) (6, 1616, 4000, 3) (6, 1616, 1)
全被试训练集,测试集，单个被试训练集，测试集形状： (4992, 4000, 3, 1) (4992, 2) (1248, 4000, 3, 1) (1248, 2) 
 (6, 6464, 4000, 3, 1) (6, 1616, 4000, 3, 1) (6, 6464, 2) (6, 1616, 2)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4000, 1, 3)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 4000, 1, 25)  1900        input_1[0][0]                    

Train on 4992 samples, validate on 1248 samples
Epoch 1/1
4992/4992 [==============================] - 430s 86ms/step - loss: 1.1621 - accuracy: 0.8253 - val_loss: 0.3269 - val_accuracy: 0.9271
(6, 3, 5000, 80)
(6, 80, 1)
数据集形状： (6, 80, 5000, 3)
训练集测试集数据及标签形状： (6, 64, 5000, 3) (6, 16, 5000, 3) (6, 64, 1) (6, 16, 1)
所有被试训练集测试集形状： (384, 5000, 3) (96, 5000, 3) (384, 1) (96, 1)
所有被试训练集测试集增强后数据形状： (4992, 4000, 3) (4992, 1) (1248, 4000, 3) (1248, 1)
每个被试增强后数据数据形状: (6, 6464, 4000, 3) (6, 6464, 1) (6, 1616, 4000, 3) (6, 1616, 1)
全被试训练集,测试集，单个被试训练集，测试集形状： (4992, 4000, 3, 1) (4992, 2) (1248, 4000, 3, 1) (1248, 2) 
 (6, 6464, 4000, 3, 1) (6, 1616, 4000, 3, 1) (6, 6464, 2) (6, 1616, 2)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 4000, 1, 3)   0                                            
___

Train on 4992 samples, validate on 1248 samples
Epoch 1/1
 432/4992 [=>............................] - ETA: 6:56 - loss: 1.6492 - accuracy: 0.7731